In [1]:
import pandas as pd
import numpy as np

In [1]:
%run loading_data.ipynb

audio : 55
audio_only : 33


In [4]:
audio_features = pd.read_csv('./data/audio_features.csv', sep='§', engine='python', index_col=0, encoding='utf-8')
audio_features.head()

,SCENE,SR,SNR,CONF,RECON,SCORE,XML
0,100_1,0.634000,0.952384,0.889849,Bonsoir intérêt pour coiffeur c'est pas sérieu...,0.573276,Bonsoir Tu veux pas te garer Pour quoi faire M...
1,107_7,0.944506,0.998807,0.882336,pourquoi trouver la je vous en prie je vous in...,0.943565,Qu est ce qui se passe Vous en faites un bouca...
2,108_3,0.683847,0.934093,0.886411,pourrais-tu vous êtes totalement à côté de la ...,0.465793,Quoi Une chorale Oui Mon pauvre Mathieu vous ê...
3,128_5,0.689977,0.991269,0.876096,marquer comme les mises en clair normal en ham...,0.596859,T as remarqué comme les nuits sont claires Nor...
4,13_2,0.855560,0.702674,0.912446,dites-moi Monsieur pires la Lombardie vous ave...,0.674074,Dites moi monsieur Pire Tina Lombardi vous avi...


In [5]:
selected_columns = ['SCENE','SR','SNR','CONF','SCORE']

In [8]:
TEMP = audio[['doc','moyenne']].groupby(['doc']).agg(['count','mean']).droplevel(0,axis=1)
RES = pd.merge(audio_features[selected_columns],TEMP, left_on='SCENE', right_on='doc').sort_values(by=['SCENE'])
RES = RES.reset_index(drop=True)
RES.head()

,SCENE,SR,SNR,CONF,SCORE,count,mean
0,100_1,0.634000,0.952384,0.889849,0.573276,2,57.583333
1,107_7,0.944506,0.998807,0.882336,0.943565,2,60.500000
2,108_3,0.683847,0.934093,0.886411,0.465793,2,49.833333
3,116_15,0.869995,0.998339,0.903309,0.364629,2,34.250000
4,128_5,0.689977,0.991269,0.876096,0.596859,2,82.250000
